In [ ]:
import tkinter as tk
from tkinter import messagebox
import random

board = [""] * 9
move_stack = []
player_score = 0
computer_score = 0
current_turn = "X"

# UI
BG = "#0f1720"
BTN = "#1f2937"
ACCENT = "#06b6d4"
TEXT = "#e6eef3"
WIN_HIGHLIGHT = "#16a34a"

# Winning lines
WIN_COMBOS = [
    (0,1,2),(3,4,5),(6,7,8),
    (0,3,6),(1,4,7),(2,5,8),
    (0,4,8),(2,4,6)
]


root = tk.Tk()
root.title("Tic-Tac-Toe")
root.configure(bg=BG)
root.resizable(False, False)

current_mode = tk.StringVar(master=root)
current_mode.set("Vs Computer")
difficulty = tk.StringVar(master=root)
difficulty.set("Hard")
starter = tk.StringVar(master=root)
starter.set("Player")


buttons = []


# Game logic funcs
def check_winner(brd):
    for a,b,c in WIN_COMBOS:
        if brd[a] == brd[b] == brd[c] and brd[a] != "":
            return brd[a], (a,b,c)
    if "" not in brd:
        return "Draw", None
    return None, None

def minimax(brd, depth, is_maximizing):
    winner, _ = check_winner(brd)
    if winner == "O":
        return 10 - depth
    elif winner == "X":
        return depth - 10
    elif winner == "Draw":
        return 0

    if is_maximizing:
        best = -999
        for i in range(9):
            if brd[i] == "":
                brd[i] = "O"
                score = minimax(brd, depth+1, False)
                brd[i] = ""
                if score > best:
                    best = score
        return best
    else:
        best = 999
        for i in range(9):
            if brd[i] == "":
                brd[i] = "X"
                score = minimax(brd, depth+1, True)
                brd[i] = ""
                if score < best:
                    best = score
        return best

def ai_choose_move():
    if difficulty.get() == "Easy":
        empties = [i for i, v in enumerate(board) if v == ""]
        return random.choice(empties) if empties else None

    # Hard: minimax with tie-breaking randomness
    best_score = -999
    moves = []
    for i in range(9):
        if board[i] == "":
            board[i] = "O"
            score = minimax(board, 0, False)
            board[i] = ""
            if score > best_score:
                best_score = score
                moves = [i]
            elif score == best_score:
                moves.append(i)
    return random.choice(moves) if moves else None

def mark_emoji(mark):
    return "❌" if mark == "X" else "🤖"

def animate_win(combo):
    flashes = 6
    def flash(count):
        if count == 0:
            for i in combo:
                buttons[i].config(bg=BTN)
            return
        color = WIN_HIGHLIGHT if count % 2 == 0 else ACCENT
        for i in combo:
            buttons[i].config(bg=color)
        root.after(160, lambda: flash(count-1))
    flash(flashes)


# UI Actions

def handle_post_move():
    global current_turn, player_score, computer_score
    winner, win_combo = check_winner(board)
    if winner:
        if winner == "X":
            player_score += 1
            player_score_label.config(text=f"Player: {player_score}")
            if win_combo:
                animate_win(win_combo)
            root.after(220, lambda: messagebox.showinfo("Result", "Player ❌ wins!"))
            root.after(300, reset_round)
            return
        elif winner == "O":
            computer_score += 1
            computer_score_label.config(text=f"Computer: {computer_score}")
            if win_combo:
                animate_win(win_combo)
            root.after(220, lambda: messagebox.showinfo("Result", "Computer 🤖 wins!"))
            root.after(300, reset_round)
            return
        else:
            root.after(120, lambda: messagebox.showinfo("Result", "It's a Draw."))
            root.after(200, reset_round)
            return

    # switch turns
    current_turn = "O" if current_turn == "X" else "X"
    update_turn_label()

    # If AI mode and it's O's turn, schedule AI move
    if current_mode.get() == "Vs Computer" and current_turn == "O":
        robot_think_animation()

def make_move(idx):
    global current_turn
    if board[idx] != "":
        return
    board[idx] = current_turn
    move_stack.append((idx, current_turn))
    buttons[idx].config(text=mark_emoji(current_turn), state="disabled", fg=TEXT)
    handle_post_move()

def make_move_button(i):
    if current_mode.get() == "Hotseat":
        if board[i] == "":
            make_move(i)
    else:
        # Vs Computer: only allow X (player) to place when it's X's turn
        if current_turn == "X" and board[i] == "":
            make_move(i)

def robot_think_animation():
    robot_label.config(text="🤖 Thinking...")
    root.update()
    root.after(550, perform_ai_move)

def perform_ai_move():
    robot_label.config(text="")
    idx = ai_choose_move()
    if idx is not None:
        board[idx] = "O"
        move_stack.append((idx, "O"))
        buttons[idx].config(text=mark_emoji("O"), state="disabled", fg=TEXT)
        handle_post_move()

def reset_round():
    global board, move_stack, current_turn
    board = [""] * 9
    move_stack = []
    for b in buttons:
        b.config(text="", state="normal", bg=BTN)
    current_turn = "X" if starter.get() == "Player" else "O"
    update_turn_label()
    if current_mode.get() == "Vs Computer" and current_turn == "O":
        root.after(400, robot_think_animation)

def reset_game():
    global player_score, computer_score
    player_score = 0
    computer_score = 0
    player_score_label.config(text=f"Player: {player_score}")
    computer_score_label.config(text=f"Computer: {computer_score}")
    reset_round()

def undo_move():
    global current_turn
    if not move_stack:
        return
    idx, mark = move_stack.pop()
    board[idx] = ""
    buttons[idx].config(text="", state="normal", bg=BTN)
    # Restore turn to the player who made that move (so they can play again)
    current_turn = mark
    update_turn_label()

def update_turn_label():
    if current_mode.get() == "Hotseat":
        if current_turn == "X":
            turn_text = "Turn: Player 1 ❌"
        else:
            turn_text = "Turn: Player 2 🤖"
    else:
        if current_turn == "X":
            turn_text = "Turn: Player ❌"
        else:
            turn_text = "Turn: Computer 🤖"
    turn_label.config(text=turn_text)

def on_mode_change():
    reset_round()

# Build the UI

top = tk.Frame(root, bg=BG, pady=8)
top.pack(fill="x")

title = tk.Label(top, text="🤖 O_X Genius", font=("Segoe UI", 18, "bold"), bg=BG, fg=ACCENT)
title.pack()

controls = tk.Frame(root, bg=BG, pady=6)
controls.pack(fill="x", padx=8)

# Mode selector
tk.Label(controls, text="Mode:", bg=BG, fg=TEXT).grid(row=0, column=0, sticky="w")
tk.Radiobutton(controls, text="Vs Computer", variable=current_mode, value="Vs Computer",
               command=on_mode_change, bg=BG, fg=TEXT, selectcolor=BG, activebackground=BG).grid(row=0, column=1, sticky="w")
tk.Radiobutton(controls, text="Hotseat (2 players)", variable=current_mode, value="Hotseat",
               command=on_mode_change, bg=BG, fg=TEXT, selectcolor=BG, activebackground=BG).grid(row=0, column=2, sticky="w")

# Difficulty
tk.Label(controls, text="Difficulty:", bg=BG, fg=TEXT).grid(row=1, column=0, sticky="w")
tk.Radiobutton(controls, text="Easy", variable=difficulty, value="Easy", bg=BG, fg=TEXT, selectcolor=BG).grid(row=1, column=1, sticky="w")
tk.Radiobutton(controls, text="Hard", variable=difficulty, value="Hard", bg=BG, fg=TEXT, selectcolor=BG).grid(row=1, column=2, sticky="w")

# Who starts
tk.Label(controls, text="Starter:", bg=BG, fg=TEXT).grid(row=2, column=0, sticky="w")
tk.Radiobutton(controls, text="Player", variable=starter, value="Player", bg=BG, fg=TEXT, selectcolor=BG).grid(row=2, column=1, sticky="w")
tk.Radiobutton(controls, text="Computer", variable=starter, value="Computer", bg=BG, fg=TEXT, selectcolor=BG).grid(row=2, column=2, sticky="w")

# Board
board_frame = tk.Frame(root, bg=BG, pady=10)
board_frame.pack(padx=10)

for i in range(9):
    btn = tk.Button(board_frame, text="", font=("Segoe UI", 28), width=4, height=2,
                    bg=BTN, fg=TEXT,
                    command=lambda i=i: make_move_button(i))
    btn.grid(row=i//3, column=i%3, padx=6, pady=6)
    buttons.append(btn)

# Bottom controls & labels
bottom = tk.Frame(root, bg=BG, pady=8)
bottom.pack(fill="x", padx=8)

turn_label = tk.Label(bottom, text="Turn: Player ❌", bg=BG, fg=TEXT, font=("Segoe UI", 12))
turn_label.pack()

robot_label = tk.Label(bottom, text="", bg=BG, fg=ACCENT, font=("Segoe UI", 10, "italic"))
robot_label.pack()

scores = tk.Frame(root, bg=BG, pady=6)
scores.pack()

player_score_label = tk.Label(scores, text=f"Player: {player_score}", bg=BG, fg=TEXT, font=("Segoe UI", 12))
player_score_label.grid(row=0, column=0, padx=10)
computer_score_label = tk.Label(scores, text=f"Computer: {computer_score}", bg=BG, fg=TEXT, font=("Segoe UI", 12))
computer_score_label.grid(row=0, column=1, padx=10)

controls2 = tk.Frame(root, bg=BG, pady=6)
controls2.pack()

reset_btn = tk.Button(controls2, text="Reset Round", command=reset_round, bg=BTN, fg=TEXT)
reset_btn.grid(row=0, column=0, padx=6)

reset_game_btn = tk.Button(controls2, text="Reset Game (scores)", command=reset_game, bg=BTN, fg=TEXT)
reset_game_btn.grid(row=0, column=1, padx=6)

undo_btn = tk.Button(controls2, text="Undo", command=undo_move, bg=BTN, fg=TEXT)
undo_btn.grid(row=0, column=2, padx=6)

# Keyboard shortcuts: 1-9 for board positions, u for undo, r for reset round
def on_key(event):
    key = event.char
    if key in "123456789":
        idx = int(key) - 1
        make_move_button(idx)
    elif key.lower() == "u":
        undo_move()
    elif key.lower() == "r":
        reset_round()

root.bind("<Key>", on_key)

# Initialize first round based on starter & mode
def initial_setup():
    for b in buttons:
        b.config(text="", state="normal", bg=BTN)
    for i in range(9):
        board[i] = ""
    move_stack.clear()
    global current_turn
    current_turn = "X" if starter.get() == "Player" else "O"
    update_turn_label()
    if current_mode.get() == "Vs Computer" and current_turn == "O":
        root.after(600, robot_think_animation)

# Watch for changes
starter.trace_add("write", lambda *args: initial_setup())
current_mode.trace_add("write", lambda *args: initial_setup())

initial_setup()

footer = tk.Label(root, text="Theme: Dark • Robot Arena • Emoji Icons  • Undo available (single-step)", bg=BG, fg="#94a3b8", font=("Segoe UI", 9))
footer.pack(pady=(6,12))

root.mainloop()
